# PointNet for particle flow

<div class="alert alert-block alert-success">
This notebook showcases the sweep functionality offered by W&B.

Sweeps are useful to to automate hyperparameter search and visualize rich, interactive experiment tracking. They basically work as a scheduler of several experiments in fixed conditions, but varying for a few sweep parameters.

Hence, we can use sweeps defining a some hyperparameters to be tuned and a grid/distribution for possible values to attempt, and the wandb UI takes care of spawning several training jobs for different values of the hyperparameters.

How the values of each experiment are set can be chosen from three different sampling schema:
 - grid search
 - random search
 - Bayesian optimization

**Main changes:** 

- repeat full pipeline for different learning rates exploiting wandb sweeps

</div>

## Problem

This dataset contains a Monte Carlo simulation of $\rho^{\pm} \rightarrow \pi^{\pm} + \pi^0$ decays and the corresponding detector response. Specifically, the data report the measured response of **i) tracker** and **ii) calorimeter**, along with the true pyshical quantitites that generated those measurements.

<div class="alert alert-block alert-info">
This means that we expect one track per event, with mainly two energy blobs (clusters of cells) in the calorimeter.
</div>

The final **goal** is to associate the cell signals observed in the calorimeter to the track that caused those energy deposits.

## Method

The idea is to leverage a **point cloud** data representation to combine tracker and calorimeter information so to associate cell hits to the corresponding track. We will use a [**PointNet**](https://openaccess.thecvf.com/content_cvpr_2017/papers/Qi_PointNet_Deep_Learning_CVPR_2017_paper.pdf) model that is capable of handling this type of data, framed as a **semantic segmentation** approach. More precisely, this means that:
- we represent each hit in the detector as a point in the point cloud: x, y, z coordinates + additional features ("3+"-dimensional point)
- the **learning task** will be binary classification at hit level: for each cell the model learns whether its energy comes mostly from the track (class 1) or not (class 0)

## Data structure

<div class="alert alert-block alert-info">

This dataset is organized as follows:
 - for each event, we create a **sample** (i.e. point cloud)
 - each sample contains all hits in a cone around a track of the event, called **focal track**
     - the cone includes all hits within some $\Delta R$ distance of the track
     - if an event has multiple tracks, then we have more samples per event
     - since different samples have possibly different number of hits, **we pad all point clouds to ensure they have same size** (needed since the model requires inputs of same size)

</div>

## Settings & config

This section collects all configuration variables and training/model hyperparameters. 

The idea is to put it at the top so that it is easy to find and edit.

In [1]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt

# path settings
REPO_BASEPATH = Path().cwd().parent
DATA_PATH = REPO_BASEPATH / "pnet_data/raw/rho_small.npz"
CODE_PATH = REPO_BASEPATH / "src"
sys.path.append(str(CODE_PATH))
MODEL_CHECKPOINTS_PATH = REPO_BASEPATH / "results" / "models" / "pointnet_baseline.weights.h5"

import wandb
from data_viz import *
from model_utils import *

LABELS = ["unfocus hit", "focus hit"]

# set random seed for reproducibility
SEED = 18
set_global_seeds(SEED)

# data settings
N_TRAIN, N_VAL, N_TEST = 210, 65, 50 # roughly 0.65, 0.2, 0.15

# model settings
N_FEATURES = 3
INIT_SIZE = 8
END_SIZE = 16

# training settings
BATCH_SIZE = 16
EPOCHS = 20
INIT_LR = 0.003

2024-11-26 15:02:01.204243: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 15:02:03.433660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Model training

We proceed with model training:

1. split the data
1. build our PointNet model using Tensorflow/Keras
1. create a dataloader to feed batches into our model
1. train
1. check results

### PointNet model 

We use a PointNet model for semantic segmentation. Here is an illustration of its structure:

![PointNet architecture](../pnet_data/images/pointnet-architecture.jpg)

We have two heads:
 - classification head (used for point cloud classification)
 - segmentation head (used for semantic segmentation)

We are going to use the **segmentation head** for our problem. The architecture settings we can experiment with are:
 - `n_features` (the number of input features): original version has only size 3 as it only takes x,y,z coordinates
 - `init_size` (number of filters of first convolutional layer): original version has 64
 - `end_size` (number of filters in segmentation head): original version has 128

In [19]:
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

def read_data(split, bin_cutoff=0.5, n_classes=2):
    split_data_path = DATA_PATH.parent.parent
    filepath=str(split_data_path / f"{split}_data" / DATA_PATH.name)
    data = np.load(filepath)['feats']
    target_class = [(energy_fraction > bin_cutoff).astype(np.float32) 
                    for energy_fraction in data['truth_cell_focal_fraction_energy']]
    # target_class = (events["truth_cell_focal_fraction_energy"] > 0.5).reshape(-1)
    target_class = keras.utils.to_categorical(target_class, num_classes=n_classes)
    return data, target_class
    

def train_wrapper():
    run = wandb.init(project="mlops-ai_infn", entity="lclissa", name="first-sweep",
                job_type="sweep", notes="Playing with sweeps ...")
    cfg = run.config
    input_features = ["normalized_x", "normalized_y", "normalized_z"]

    train_data, train_label_cloud = read_data("train")
    
    train_point_clouds = train_data[input_features]
    total_training_examples = len(train_point_clouds)
    
    val_data, val_label_cloud = read_data("val")
    val_point_clouds = val_data[input_features]
    
    print("Num train point clouds:", len(train_point_clouds))
    print("Num train point cloud labels:", len(train_label_cloud))
    print("Num val point clouds:", len(val_point_clouds))
    print("Num val point cloud labels:", len(val_label_cloud))
    
    n_points = train_point_clouds[0].shape[0]
    n_features = len(train_point_clouds[0].dtype.names)
    n_classes = len(LABELS)

    _ = run.use_artifact("train_data:latest")
    _ = run.use_artifact("val_data:latest")
    
    train_dataset = generate_dataset(train_point_clouds, train_label_cloud, 
                                 bs=cfg.batch_size, n_points=n_points, n_features=n_features, labels=LABELS)
    val_dataset = generate_dataset(val_point_clouds, val_label_cloud, is_training=False, 
                                   bs=cfg.batch_size, n_points=n_points, n_features=n_features, labels=LABELS)

    
    steps_per_epoch = total_training_examples // cfg.batch_size
    total_training_steps = steps_per_epoch * EPOCHS
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=cfg.init_lr,
        decay_steps=steps_per_epoch * 5,
        decay_rate=0.5,
        staircase=True,
    )

    segmentation_model = get_shape_segmentation_model(n_points, n_classes, n_features,
                                                      INIT_SIZE, END_SIZE)
    segmentation_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
        jit_compile=False
    )

    MODEL_CHECKPOINTS_PATH.parent.mkdir(exist_ok=True, parents=True)
    history = segmentation_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[
            WandbMetricsLogger(log_freq=5),
            WandbModelCheckpoint(
                       MODEL_CHECKPOINTS_PATH, #.parent / "model-{epoch:02d}-{val_loss:.2f}.weights.h5",
                       monitor="val_loss",
                       save_best_only=True,
                       save_weights_only=True,
                   )
        ],
    )
    

In [20]:
# 2: Define the search space
sweep_configuration = {
    "method": "bayes",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "init_lr": {"max": 0.1, "min": 0.01},
        "batch_size": {"values": [16, 32]},
    },
}

# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project="mlops-ai_infn", entity="lclissa")

wandb.agent(sweep_id, function=train_wrapper, count=5)

Create sweep with ID: 59n7uolz
Sweep URL: https://wandb.ai/lclissa/mlops-ai_infn/sweeps/59n7uolz


wandb: Agent Starting Run: vnv2uawp with config:
wandb: 	batch_size: 32
wandb: 	init_lr: 0.01777628046019689
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65


wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


Epoch 1/20


...

Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65
Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 215ms/step - accuracy: 0.5987 - loss: 42.1127 - val_accuracy: 0.8751 - val_loss: 30.7271
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.8575 - loss: 30.1240 - val_accuracy: 0.8734 - val_loss: 27.4938
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.8701 - loss: 19.8802 - val_accuracy: 0.8734 - val_loss: 24.1101
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.8727 - loss: 18.2878 - val_accuracy: 0.8734 - val_loss: 21.6381
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.8791 - loss: 18.3399 - val_accuracy: 0.8734 - val_loss: 20.5214
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.8796 - loss: 17.9241 - val_accuracy: 0.8734 - val_loss: 19.9021
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.8755 - loss: 17.7300 - val_accuracy: 0.8734 - val_

batch/accuracy,▁▅▇▇▇▇▇▇██▇█▇▇██▇▇█▇████▇████▇▇▇██▇█▇███
batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,█████████▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▆▇█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▇▇█████████████████
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,████▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁▁
epoch/loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.88153


wandb: Agent Starting Run: erus557b with config:
wandb: 	batch_size: 32
wandb: 	init_lr: 0.055696311338659446
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65
Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 214ms/step - accuracy: 0.8448 - loss: 4142.1177 - val_accuracy: 0.8734 - val_loss: 2824019.5000
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.8731 - loss: 838.8868 - val_accuracy: 0.8734 - val_loss: 480720715776.0000
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.8757 - loss: 471.7996 - val_accuracy: 0.8734 - val_loss: 43942594740224.0000
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.8847 - loss: 250.2328 - val_accuracy: 0.1266 - val_loss: 47312365682688.0000
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.8888 - loss: 145.2924 - val_accuracy: 0.1266 - val_loss: 2824353611776.0000
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.8772 - loss: 116.2681 - val_accuracy: 0.1266 - val_loss: 143078244352.0000
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - a

batch/accuracy,▁▅▅▆▇▆▇▆█▆▅▆▆▆▆▆▅▆▆▆▆▆▆▆█▆▆▆▅▆▅▆▆▆▅▆▅▆▆▆
batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,█████████▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▁█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▇▇▇▇▇▇▇▆█▇▇█▇▇▇▇▇▇▇
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,████▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁▁
epoch/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,███▁▁▁▁▁▁▁▁▂▇███████
epoch/val_loss,▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.87909


<div class="alert alert-block alert-info">

For more info check [wandb.sweeps](https://docs.wandb.ai/guides/sweeps/)
</div>